In [12]:
import escher
import modelseedpy
import modelseedpy_escher

In [4]:
manager = modelseedpy_escher.EscherManager(escher)

In [9]:
em = manager.get_map('Escherichia coli', 'e_coli_core', 'Core metabolism')

In [11]:
from modelseedpy_escher.map.model_merge import RefitMap

In [25]:
refit = RefitMap2(None)

In [26]:
refit.refit(em, {})

AttributeError: 'NoneType' object has no attribute 'get_seed_reaction'

In [21]:
for node in em.nodes:
    node
node

{'y': 1041.710205078125,
 'x': 1056.5867919921875,
 'node_type': 'multimarker',
 'uid': '1576999'}

In [24]:
class RefitMap2:

    def __init__(self, modelseed):
        self.ms = modelseed
        self.updated_node = set()

    def refit_map_metabolite(self, o, cmp_token):
        if o['bigg_id'] not in self.updated_node:
            o['bigg_id'] = "{}_{}".format(o['bigg_id'], cmp_token)
            o['name'] = "{} [{}]".format(o['name'], cmp_token)
            self.updated_node.add(o['bigg_id'])

    def refit_map_reaction(self, o, nodes, cmp_config):
        cmp_token = cmp_config['0']
        # print('refit_map_reaction', o['bigg_id'], cmp_config, cmp_token, len(cmp_token))
        if len(cmp_token) > 0:
            for s_uid in o['segments']:
                segment = o['segments'][s_uid]
                from_node = nodes[segment['from_node_id']]
                to_node = nodes[segment['to_node_id']]
                if to_node['node_type'] == 'metabolite':
                    self.refit_map_metabolite(to_node, cmp_token)
                if from_node['node_type'] == 'metabolite':
                    self.refit_map_metabolite(from_node, cmp_token)

            cmp_func = lambda x: {'bigg_id': x['bigg_id'] + '_' + cmp_token, 'coefficient': -1 * x['coefficient']}
            o['metabolites'] = list(map(cmp_func, o['metabolites']))

            o['bigg_id'] = "{}_{}".format(o['bigg_id'], cmp_token)
            o['name'] = "{} [{}]".format(o['name'], cmp_token)

    @staticmethod
    def reverse(map_reaction):
        rev_func = lambda x: {'bigg_id': x['bigg_id'], 'coefficient': -1 * x['coefficient']}
        metabolites = list(map(rev_func, map_reaction['metabolites']))
        map_reaction['metabolites'] = metabolites
        return map_reaction

    def refit(self, em, cmp_config):
        em = em.clone()
        self.updated_node = set()
        nodes = dict(map(lambda x: (x['uid'], x), em.nodes))
        seed_ids = {}
        for o in em.reactions:
            seed_id = o['bigg_id']
            rxn = self.ms.get_seed_reaction(seed_id)
            if rxn is not None:
                seed_ids[rxn.id] = rxn
                if rxn.data['direction'] == '=':
                    o['reversibility'] = True

                map_stoich = dict(map(lambda x: ((x['bigg_id'], '0'), x['coefficient']), o['metabolites']))
                cstoichiometry = rxn.cstoichiometry
                if rxn.data['direction'] == '<':
                    cstoichiometry = dict(map(lambda x: (x[0], -1 * x[1]), cstoichiometry.items()))
                if not map_stoich == cstoichiometry:
                    self.reverse(o)
                self.refit_map_reaction(o, nodes, cmp_config)
                if 'annotation' not in o:
                    o['annotation'] = {}
                if 'seed.reaction' not in o['annotation']:
                    o['annotation']['seed.reaction'] = []
                if type(o['annotation']['seed.reaction']) == str:
                    o['annotation']['seed.reaction'] = [o['annotation']['seed.reaction']]
                if rxn.id not in o['annotation']['seed.reaction']:
                    o['annotation']['seed.reaction'].append(rxn.id)
                o['annotation']['seed.compartment'] = ';'.join(map(lambda x: x[0] + ':' + x[1], cmp_config.items()))
        return em